###Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

- Наименование вакансии.
- Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.

###По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import json
import re
import pandas as pd
import numpy as np


In [2]:
def _parser_hh(vacancy):

    vacancy_date = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36'
    }

    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = 1
        else:
            last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())
    
    for page in range(last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_date.append(_parser_item_hh(item))
                
    return vacancy_date


In [3]:
def _parcer_item_hh(item):

  vacancy_date = {}

# Наименование позиции
  vacancy_name = item.find('div', {'class': 'resume-search-item__name'}) \
                        .getText() \
                        .replace(u'\xa0', u' ')
  vacancy_date['vacancy_name'] = vacancy_name

# Работодатель
  company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info'}) \
                        .find('a') \
                        .getText()
    
  vacancy_date['company_name'] = company_name
# Город
  city = item.find('span', {'class' : 'vacancy-serp-item__meta-info'}) \
                        .getText() \
                        .split(', ')[0]
  vacancy_date['city'] = city

# Зароботная плата

  salary = item.find('div', {'class': 'vacancy-serp-item__compensation'})
  if not salary:
    salary_min = None
    salary_max = None
    salary_currency = None
  else:
    salary = salary.getText() \
                    .replace(u'\xa0', u'')
    salary = re.split(r'\s|-', salary)

    if salary[0] == 'до':
      salary_min = None
      salary_max = int(salary[1])
    elif salary[0] == 'от':
      salary_min = int(salary[1])
      salary_max = None
    else:
      salary_min = int(salary[0])
      salary_max = int(salary[1])
    salary_currency = salary[2]

  vacancy_date['salary_min'] = salary_min
  vacancy_date['salary_max'] = salary_max
  vacancy_date['salary_currency'] = salary_currency

# Ссылка
  is_ad = item.find('sapn', {'class' : 'vacancy-serp-item__controls-item vacancy-serp-item__controls-item_last'}) \
              .getText()
  vacancy_link = item.find('div', {'class' : 'resume-search-item__name'}) \
                      .find('a')['href']

  if is_ad != 'Реклама':
      vacancy_link = vacancy_link.split('?')[0]

  vacancy_date['site'] = 'hh.ru'

  return vacancy_date

In [15]:
def parser_vacancy(vacancy):
    vacancy_date = []
    vacancy_date.extend(_parser_hh(vacancy))
    
    df = pd.DataFrame(vacancy_date)
    df.to_csv('hh_data.csv')
    return df